In [1]:
import json
import requests
import pandas as pd
from datetime import datetime,timedelta, date
from operator import itemgetter

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
def createOlapPayload(sources,metrics,dimensions,startDate,endDate,filters):
	data = {}
	data['sources'] = sources
	data['metrics'] = metrics
	i = 1
	dimensionsDict = {}
	for dimension in dimensions:
		dimensionsDict[dimension] = i
		i += 1
	data['dimensions'] = dimensionsDict
	date1 = datetime.strptime(startDate,'%Y-%m-%d').strftime("%Y-%m-%d")
	date2 = (datetime.strptime(endDate,'%Y-%m-%d') + timedelta(1)).strftime("%Y-%m-%d")
	segments = []
	segmentsDict = {}
	segmentsDict['dateStart'] = "{0}T00:00:00+03:00".format(date1)
	segmentsDict['dateEnd'] = "{0}T00:00:00+03:00".format(date2)
	segmentsDict['filters'] = {}
	for filterKey, filterValue in filters.iteritems():
		segmentsDict['filters'][filterKey] = {}
		segmentsDict['filters'][filterKey]['type'] = 'DIMENSION'
		segmentsDict['filters'][filterKey]['ids'] = filterValue
	segments.append(segmentsDict)
	data['segments'] = segments
	return data

def getJSONOlapResponse(data):
	url = 'http://192.168.10.5:4000/api/OLAP/Plain'
	headers = {'Content-Type': 'application/json'}
	response = requests.post(url,data,headers=headers)
	return json.loads(response.text)

def transformOlapResponse(response,payload):
	data = []
	dimensionsList = [{'key': k, 'value': v} for k,v in payload['dimensions'].iteritems()]
	dimensionsList = sorted(dimensionsList, key=itemgetter('value'))
	for item in response:
		i = 0
		dataDict = {}
		for dimensionValue in item['dimensions']:
			dimensionName = dimensionsList[i]['key']
			dataDict[dimensionName] = dimensionValue
			i += 1
		metrics = item['value'][0]
		for k,v in metrics.iteritems():
			dataDict[k] = v
		data.append(dataDict)
	return data

In [16]:
start_date = date(2016, 3, 1)
end_date = date(2016, 3, 2)

In [17]:
total_df = pd.DataFrame()
for date1 in daterange(start_date, end_date):
    print date1,
    date2 = date1 + timedelta(1)
    print '\t',date2
    filters = {'campaignTypeId': ['1','2'], 'projectId': ['2337938','2425053','2376103','2404711','2406006','2273493','2345116','2371741','2214297','2337395','2434227','2357499']}
    source = ["direct_bid","direct_conversion","metrika"]
    metrics = ["shows","clicks","spent","visits","cpaVisits","rejectPercent"]
    dimensions = ['dateDay','campaignExternalId','campaignName','adExternalId','keyPhraseName']
    startDate = date1.strftime("%Y-%m-%d")
    endDate = date2.strftime("%Y-%m-%d")
    payload = createOlapPayload(source,metrics,dimensions,startDate,endDate,filters)
    response = getJSONOlapResponse(json.dumps(payload))
    print 'Get Response'
    data = transformOlapResponse(response,payload)
    df = pd.DataFrame(data)
    total_df = pd.concat([total_df,df])

2016-03-01 	2016-03-02
Get Response
2016-03-02 	2016-03-03
Get Response
2016-03-03 	2016-03-04
Get Response
2016-03-04 	2016-03-05
Get Response
2016-03-05 	2016-03-06
Get Response
2016-03-06 	2016-03-07
Get Response
2016-03-07 	2016-03-08
Get Response
2016-03-08 	2016-03-09
Get Response
2016-03-09 	2016-03-10
Get Response
2016-03-10 	2016-03-11
Get Response
2016-03-11 	2016-03-12
Get Response
2016-03-12 	2016-03-13
Get Response
2016-03-13 	2016-03-14
Get Response
2016-03-14 	2016-03-15
Get Response
2016-03-15 	2016-03-16
Get Response
2016-03-16 	2016-03-17
Get Response
2016-03-17 	2016-03-18
Get Response
2016-03-18 	2016-03-19
Get Response
2016-03-19 	2016-03-20
Get Response
2016-03-20 	2016-03-21
Get Response
2016-03-21 	2016-03-22
Get Response
2016-03-22 	2016-03-23
Get Response
2016-03-23 	2016-03-24
Get Response
2016-03-24 	2016-03-25
Get Response
2016-03-25 	2016-03-26
Get Response
2016-03-26 	2016-03-27
Get Response
2016-03-27 	2016-03-28
Get Response
2016-03-28 	2016-03-29
Get R

In [18]:
total_df

,adExternalId,campaignExternalId,campaignName,clicks,cpaVisits,dateDay,goalVisits,keyPhraseName,rejectPercent,shows,spent,visits
0,564665598,10489147,SAAB автозапчасти,0,0,2016-03-01,0,Saab 900 запчасти,0,0,0,0
1,564665598,10489147,SAAB автозапчасти,0,0,2016-03-01,0,Saab 9000 запчасти цены,0,0,0,0
2,564665598,10489147,SAAB автозапчасти,0,0,2016-03-01,0,Saab 9000 разборка,0,0,0,0
3,564665598,10489147,SAAB автозапчасти,0,0,2016-03-01,0,Авито запчасти Сааб,0,0,0,0
4,564665598,10489147,SAAB автозапчасти,0,0,2016-03-01,0,Автозапчасти CААБ,0,0,0,0
5,564665598,10489147,SAAB автозапчасти,0,0,2016-03-01,0,Автозапчасти Saab Б +У,0,0,0,0
6,564665598,10489147,SAAB автозапчасти,0,0,2016-03-01,0,Автозапчасти Saab БУ,0,0,0,0
7,564665598,10489147,SAAB автозапчасти,0,0,2016-03-01,0,Автозапчасти Saab дешево,0,0,0,0
8,564665598,10489147,SAAB автозапчасти,0,0,2016-03-01,0,Автозапчасти Saab купить,0,0,0,0
9,564665598,10489147,SAAB автозапчасти,0,0,2016-03-01,0,Автозапчасти Saab недорого,0,0,0,0


In [19]:
from pandas import HDFStore
store = HDFStore('D:\\store2.h5')

In [22]:
store['total_df'] = total_df

C:\Users\aleksey.makarov\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0] [items->None]

  if __name__ == '__main__':
C:\Users\aleksey.makarov\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block0_items] [items->None]

  if __name__ == '__main__':


In [21]:
total_df.to_csv('D:\\bounce_rate.csv',encoding='utf-8',sep='\t')

In [23]:
store.close()

In [38]:
store = HDFStore('D:\\store.h5')

In [40]:
store.close()

In [24]:
total_df['bounces'] = total_df['visits']*total_df['rejectPercent']/100

In [25]:
total_df.columns

Index([      u'adExternalId', u'campaignExternalId',       u'campaignName',
                   u'clicks',          u'cpaVisits',            u'dateDay',
               u'goalVisits',      u'keyPhraseName',      u'rejectPercent',
                    u'shows',              u'spent',             u'visits',
                  u'bounces'],
      dtype='object')

In [26]:
grouped_df = total_df.groupby(['adExternalId','campaignExternalId','keyPhraseName','campaignName'],as_index=False).sum()

In [27]:
grouped_df = grouped_df[grouped_df['shows'] > 0]

In [28]:
grouped_df['bounce_rate'] = grouped_df['bounces']/grouped_df['visits']

In [29]:
grouped_df[(grouped_df['bounce_rate'] == 0.0)]['cpaVisits'].sum()/grouped_df[(grouped_df['bounce_rate'] == 0.0)]['visits'].sum()*100

1.1429642121041785

In [30]:
grouped_df_drop_na = grouped_df.dropna()

In [31]:
campaigns_df = grouped_df_drop_na[['campaignExternalId','visits','bounces']].groupby(['campaignExternalId'],as_index=False).sum()

In [32]:
campaigns_df['campaign_bounce_rate'] = campaigns_df['bounces']/campaigns_df['visits']

In [33]:
merged_df = pd.merge(grouped_df_drop_na,campaigns_df[['campaignExternalId','campaign_bounce_rate']],how='left',on=['campaignExternalId'])

In [34]:
ads_df = grouped_df_drop_na[['adExternalId','visits','bounces']].groupby(['adExternalId'],as_index=False).sum()

In [35]:
ads_df['ad_bounce_rate'] = ads_df['bounces']/ads_df['visits']

In [36]:
merged_df = pd.merge(merged_df,ads_df[['adExternalId','ad_bounce_rate']],how='left',on=['adExternalId'])

In [37]:
merged_df.to_excel('D:\\bounce_rate_analysis2.xlsx')

In [2]:
campaigns_ads = pd.read_csv(u'C:\\Users\\aleksey.makarov\\Desktop\\2016-04\\Анализ показателя отказов\\campaign_ads_keys.csv')

In [7]:
test_df = campaigns_ads.groupby(['adGroupId','campaignId'],as_index = False).count()

In [15]:
test_df[test_df['externalId'] > 3].groupby(['campaignId'],as_index=False).count().sort_values(by='adGroupId',ascending =False).to_clipboard()

In [38]:
len(merged_df['campaignExternalId'].unique())

117

In [39]:
len(merged_df['adExternalId'].unique())

2657

In [40]:
len(merged_df['keyPhraseName'])

4922